## dCMF
Example of running the "dcmf" module with the use provided parameters

In [1]:
import sys
sys.path.append("..")

In [2]:
import pprint
import numpy as np
import pickle as pkl
import time
import itertools
import os

In [3]:
from src.dcmf import dcmf

## Loading the sample dataset

This directory contains a sample synthetic dataset generated for the augmented setting of Fig 1(c) in the [paper](https://arxiv.org/abs/1811.11427).
You can download the sample data from [here](https://drive.google.com/open?id=1EFF_kuOIg2aYyOGZY_peX3NziqCSxxP1) and unzip it to the data directory.

In [4]:
data_dir = "../data/sample_data/"

In [5]:
#Loads the dataset into a dict
#Note: This dataset contains 5-folds for the matrix X_12 (matrix R below)
num_folds = 1
#
pp = pprint.PrettyPrinter()
print("Loading data from data_dir: ",data_dir)
U1 = pkl.load(open(data_dir+"X_13.pkl",'rb'))
U2 = pkl.load(open(data_dir+"X_14.pkl",'rb'))
V1 = pkl.load(open(data_dir+"X_26.pkl",'rb'))
W1 = pkl.load(open(data_dir+"X_53.pkl",'rb'))
R_temp_dict = {}
for fold_num in np.arange(1,num_folds+1):
    Rtrain = pkl.load(open(data_dir+'/X_12_train_fold_'+str(fold_num)+'.pkl','rb'))
    Rtrain = Rtrain
    Rtrain_idx = pkl.load(open(data_dir+'/X_12_train_idx_'+str(fold_num)+'.pkl','rb')) 
    Rtest = pkl.load(open(data_dir+'/X_12_test_fold_'+str(fold_num)+'.pkl','rb'))
    Rtest_idx = pkl.load(open(data_dir+'/X_12_test_idx_'+str(fold_num)+'.pkl','rb'))
    Rdoublets = pkl.load(open(data_dir+'/R_doublets_'+str(fold_num)+'.pkl','rb'))
    R_temp_dict[fold_num] = {"Rtrain":Rtrain,"Rtrain_idx":Rtrain_idx,"Rtest":Rtest,"Rtest_idx":Rtest_idx,"Rdoublets":Rdoublets}
#
data_dict = {"U1":U1,"U2":U2,"V1":V1,"W1":W1,"R":R_temp_dict}

Loading data from data_dir:  ../data/sample_data/


In [6]:
print("U1.shape: ",U1.shape)
print("U2.shape: ",U2.shape)
print("V1.shape: ",V1.shape)
print("W1.shape: ",W1.shape)
print("R.shape: ",data_dict['R'][1]['Rtrain'].shape)

U1.shape:  (1000, 20)
U2.shape:  (1000, 150)
V1.shape:  (2000, 250)
W1.shape:  (300, 20)
R.shape:  (1000, 2000)


## Building the required data structures

Here we construct the data structures required as input to the dcmf API

#### *entity matrix relationship graph *

- **G**: dict, keys are entity IDs and values are lists of associated matrix IDs

#### * training data*
- **X_data**: dict, keys are matrix IDs and values are (1) np.array, or (2) dict, (if this matrix is in validation set **X_val**) with validation set IDs as keys & values as np.array
- **X_meta**: dict, keys are matrix IDs and values are lists of the 2 associated entity IDs

#### *validation data*
- **X_val**: dict, keys are IDs of the matrices that are part of validation set and values are dict with validation set IDs as keys and values are (1) scipy.sparse matrix, or (2) list of triplets corresponding to the validation entries (if you would like to perform classification and measure AUC)  
**Note**: To perform K folds cross validation, use K validation sets for the corresponsing matrix/matrices. In the example below, we used a single validation set with ID "1" for each of the matrices with IDs "X1" and "X2"

In [7]:
G = {
    "e1":["X1","X2","X3"],\
    "e2":["X1","X4"],\
    "e3":["X2","X5"],\
    "e4":["X3"],\
    "e5":["X5"],\
    "e6":["X4"]}
    #"e6":["X4"]}

In [8]:
X_data = {
    "X1":{"1":data_dict['R'][1]["Rtrain"]},\
    "X2":{"1":U1},\
    "X3":U2,\
    "X4":V1,\
    "X5":W1}

In [9]:
X_meta = {
    "X1":["e1","e2"],\
    "X2":["e1","e3"],\
    "X3":["e1","e4"],\
    "X4":["e2","e6"],\
    "X5":["e5","e3"]}
    #"X5":["e5","e3"]}

In [10]:
Rtest_triplets1 = [[1,1,1],[2,2,0]]
Rtest_triplets2 = [[1,1,1],[3,3,0],[1,2,0],[0,1,0],[0,2,0],[0,3,0]]

In [11]:
X_val = {
    "X1":{"1":Rtest_triplets1},
    "X2":{"1":Rtest_triplets2}
}

#### *dCMF network construction - hyperparameters*

- **kf**: float, in the range (0,1) 
- **k**: int, entity representation or encoding size. Refer Appendix A in the [paper](https://arxiv.org/abs/1811.11427) for info about how k and kf are used in the dCMF network construction. 
- **e_actf**: str, autoencoder's encoding activation function.
- **d_actf**: str, autoencoder's decoding activation function. Supported functions are "tanh","sigma","relu","lrelu"
- **is_linear_last_enc_layer**: bool, True to set linear activation for the bottleneck/encoding generation layer 
- **is_linear_last_dec_layer**: bool, True to set linear activation for the output/decoding generation layer 
- **num_chunks**: int, number of training batches to create.

In [12]:
kf = 0.5
k = 100
e_actf = "tanh"
d_actf = "tanh"
is_linear_last_enc_layer = False
is_linear_last_dec_layer = False
num_chunks = 2

#### *Optimization/training - hyperparamteres*

- **learning_rate**: float, Adam optimizer's learning rate
- **weight_decay**: float, Adam optimizers's weight decay (L2 penalty)
- **max_epochs**: int, maximum number of training epochs at which the training stops 
- **convg_thres**: float, convergence threshold 

In [13]:
learning_rate = 0.001
weight_decay = 0.05
max_epochs = 5
convg_thres = 0.1

#### *Hyperparamteres related to pre-training*

- **is_pretrain**: bool, True for pretraining 
- **pretrain_thres**: bool, pre-training convergence thresholsd
- **max_pretrain_epochs**: int, maximum number of pre-training epochs at which the training stops

In [14]:
is_pretrain=True
pretrain_thres= 0.1
max_pretrain_epochs = 2

#### *Parameters related to validation*

- **val_metric**: str, Validation performance metric. Supported metrics: ["rmse","r@k","p@k","auc"]. Where,  
     *rmse* - Root [mean square error](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html)  
     *r@k* - Recall@k. Refer section 5.2's sub-section "Evaluation metric" in the [paper](https://arxiv.org/abs/1811.11427)      
     *p@k* - Probability@k. Refer section 5.3's sub-section "Evaluation metric" in the [paper](https://arxiv.org/abs/1811.11427)      
     *auc* - [Area under the curve](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html)
    
- **is_val_transpose**: bool, True if the reconstructed matrix has to be transposed before computing the validation performance
- **at_k**: int, the value of k if the **val_metric** is either "r@k" or "p@k"

In [15]:
val_metric = "auc"
is_val_transpose = True
at_k = 10

#### *GPU - parameters *

- **is_gpu**: bool, True if pytorch tensors storage and operations has to be done in GPU
- **gpu_ids**: str, Comma separated string of CUDA GPU ID

In [16]:
is_gpu = False
gpu_ids = "1"

#### *Instantiating the dCMF model...*
- Initializes dCMF after validating the input data and the (hyper)parameters

In [17]:
dcmf_model = dcmf(G, X_data, X_meta,\
            num_chunks=num_chunks,k=k, kf=kf, e_actf=e_actf, d_actf=d_actf,\
            learning_rate=learning_rate, weight_decay=weight_decay, convg_thres=convg_thres, max_epochs=max_epochs,\
            is_gpu=is_gpu,gpu_ids=gpu_ids,is_pretrain=is_pretrain, pretrain_thres=pretrain_thres,\
            max_pretrain_epochs=max_pretrain_epochs,X_val=X_val,val_metric=val_metric,\
            is_val_transpose=is_val_transpose, at_k=at_k,\
            is_linear_last_enc_layer=is_linear_last_enc_layer,is_linear_last_dec_layer=is_linear_last_dec_layer,num_val_sets=num_folds)

dcmf_base.__init__ - start
dcmf_base.__init__ - end
#
dCMF:
---
#
dCMF: 
#
learning_rate:  0.001
weight_decay:  0.05
convg_thres:  0.1
max_epochs:  5
isPretrain:  True
pretrain_thres:  0.1
max_pretrain_epochs:  2
num_chunks:  2
k:  100
kf:  0.5
e_actf:  tanh
d_actf:  tanh
is_gpu:  False
gpu_ids:  1
num entities:  6
num matrices:  5
num_val_sets:  1
X_val #matrices:  2
val_metric (used only if X_val #matrices > 0):  auc
at_k (used only if X_val #matrices > 0 and val_metric is r@k or p@k):  10
is_val_transpose:  True
is_linear_last_enc_layer:  False
is_linear_last_dec_layer:  False
#


#### *Fitting... *
- Performs the input transformation and network construction
- (Pre-trains and) trains the model to obtain the entity representations
- Reconstruct the input matrices using the entity representations obtained

In [18]:
dcmf_model.fit()

## fold_num:  1  ##
dcmf_base.__init__ - start
dcmf_base.__init__ - end
#
dCMF: 
#
learning_rate:  0.001
weight_decay:  0.05
convg_thres:  0.1
max_epochs:  5
isPretrain:  True
pretrain_thres:  0.1
max_pretrain_epochs:  2
num_chunks:  2
k:  100
kf:  0.5
e_actf:  tanh
d_actf:  tanh
is_gpu:  False
gpu_ids:  1
num entities:  6
num matrices:  5
num_val_sets:  1
X_val #matrices:  2
val_metric (used only if X_val #matrices > 0):  auc
at_k (used only if X_val #matrices > 0 and val_metric is r@k or p@k):  10
is_val_transpose:  True
is_linear_last_enc_layer:  False
is_linear_last_dec_layer:  False
#
dcmf - model construction - start
__input_transformation - start
#
concatenated-matrix construction...
e_id:  e1
X_id_list:  ['X1', 'X2', 'X3']
X_id:  X1
X[X_id].shape:  (1000, 2000)
X_id:  X2
X[X_id].shape:  (1000, 20)
X_id:  X3
X[X_id].shape:  (1000, 150)
C_dict[e].shape:  torch.Size([1000, 2170])
---
e_id:  e2
X_id_list:  ['X1', 'X4']
X_id:  X1
X[X_id].shape:  (1000, 2000)
X_id:  X4
X[X_id].shape:

#### *Result attributes:*
- **out_dict_U**:  dict, keys are validation set IDs and values are dict with entity IDs as keys and np.array of entity representations/encodings as values
- **out_dict_X_prime**: dict, keys are matrix IDs and values are matrix reconstructions
- **out_dict_info**: dict, keys are loss/validation performance attributes and values are corresponding results.

In [19]:
dcmf_model.out_dict_U['1'].keys()

dict_keys(['e1', 'e2', 'e3', 'e4', 'e5', 'e6'])

In [20]:
dcmf_model.out_dict_X_prime['1'].keys()

dict_keys(['X1', 'X2', 'X3', 'X4', 'X5'])

In [21]:
dcmf_model.out_dict_info

{'E': 6,
 'M': 5,
 'loss_all_folds': {'1': [0.6764270067214966,
   0.7010240256786346,
   1.0403445959091187,
   0.9245219826698303,
   0.5641582608222961,
   0.9195870459079742,
   0.8119459450244904,
   0.13825598172843456,
   1.1052338425070047,
   0.37322503700852394,
   1.556600421667099]},
 'loss_all_folds_avg_sum': 8.811324145644903,
 'loss_all_folds_avg_tuple': [0.6764270067214966,
  0.7010240256786346,
  1.0403445959091187,
  0.9245219826698303,
  0.5641582608222961,
  0.9195870459079742,
  0.8119459450244904,
  0.13825598172843456,
  1.1052338425070047,
  0.37322503700852394,
  1.556600421667099],
 'num_val_sets': 1,
 'params': {'convg_thres': 0.1,
  'd_actf': 'tanh',
  'e_actf': 'tanh',
  'is_linear_last_dec_layer': False,
  'is_linear_last_enc_layer': False,
  'is_pretrain': True,
  'k': 100,
  'kf': 0.5,
  'learning_rate': 0.001,
  'max_epochs': 5,
  'max_pretrain_epochs': 2,
  'num_chunks': 2,
  'pretrain_thres': 0.1,
  'weight_decay': 0.05},
 'val_metric': 'auc',
 'val_p